## 2.1 텐서 다루기

## 2.1.1 리스트 포맷으로 pytorch tensor 만들기

In [1]:
import torch

print(torch.tensor([[1, 2], [3, 4]]))
print('------------------------')
print(torch.tensor([[1, 2], [3, 4]], device="cuda:0"))
print('------------------------')
print(torch.tensor([[1, 2], [3, 4]], dtype=torch.float64))

tensor([[1, 2],
        [3, 4]])
------------------------
tensor([[1, 2],
        [3, 4]], device='cuda:0')
------------------------
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


## 2.1.2 pytorch tensor to numpy array

In [2]:
temp = torch.tensor([[1, 2], [3, 4]])
print(temp.numpy())
print('------------------------')
temp = torch.tensor([[1, 2], [3, 4]], device="cuda:0")  #GPU가 없다면 오류가 발생하므로 주석 처리하였습니다.
#temp = torch.tensor([[1,2],[3,4]], device="cpu:0")
print(temp.to("cpu").numpy())

[[1 2]
 [3 4]]
------------------------
[[1 2]
 [3 4]]


## 2.1.3 float type 텐서

In [3]:
temp = torch.FloatTensor([1, 2, 3, 4, 5, 6, 7])
print(temp[0], temp[1], temp[-1])
print('------------------------')
print(temp[2:5], temp[4:-1])

tensor(1.) tensor(2.) tensor(7.)
------------------------
tensor([3., 4., 5.]) tensor([5., 6.])


## 2.1.4 텐서 연산

In [4]:
v = torch.tensor([1, 2, 3])
w = torch.tensor([3, 4, 6])
print(w - v)

tensor([2, 2, 3])


##  2.1.5 텐서 reshape

In [5]:
temp = torch.tensor([
    [1, 2], [3, 4]
])

print(temp.shape)
print('------------------------')
print(temp.view(4, 1))
print('------------------------')
print(temp.view(-1))
print('------------------------')
print(temp.view(1, -1))
print('------------------------')
print(temp.view(-1, 1))

torch.Size([2, 2])
------------------------
tensor([[1],
        [2],
        [3],
        [4]])
------------------------
tensor([1, 2, 3, 4])
------------------------
tensor([[1, 2, 3, 4]])
------------------------
tensor([[1],
        [2],
        [3],
        [4]])


# 2.2 파이토치 코드 맛보기

In [6]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
dataset = pd.read_csv(r'../chap02/data/car_evaluation.csv')

In [8]:
dataset.head()

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [9]:
# fig_size = plt.rcParams["figure.figsize"]
# fig_size[0] = 8
# fig_size[1] = 6
# plt.rcParams["figure.figsize"] = fig_size
# dataset.output.value_counts().plot(kind='pie', autopct='%0.05f%%', colors=['lightblue', 'lightgreen', 'orange', 'pink'],
#                                    explode=(0.05, 0.05, 0.05, 0.05))

In [10]:
categorical_columns = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']

In [11]:
for category in categorical_columns:
    dataset[category] = dataset[category].astype('category')

In [12]:
'''
pd.DataFrame.cat.code.values : 데이터의 종류에 정수로 표현
'''

price = dataset['price'].cat.codes.values
maint = dataset['maint'].cat.codes.values
doors = dataset['doors'].cat.codes.values
persons = dataset['persons'].cat.codes.values
lug_capacity = dataset['lug_capacity'].cat.codes.values
safety = dataset['safety'].cat.codes.values

categorical_data = np.stack([price, maint, doors, persons, lug_capacity, safety], 1)
categorical_data[:10]

array([[3, 3, 0, 0, 2, 1],
       [3, 3, 0, 0, 2, 2],
       [3, 3, 0, 0, 2, 0],
       [3, 3, 0, 0, 1, 1],
       [3, 3, 0, 0, 1, 2],
       [3, 3, 0, 0, 1, 0],
       [3, 3, 0, 0, 0, 1],
       [3, 3, 0, 0, 0, 2],
       [3, 3, 0, 0, 0, 0],
       [3, 3, 0, 1, 2, 1]], dtype=int8)

In [13]:
'''
데이터 프레임을 파이토치 텐서로 바꿈
torch.tensor(pd.Dataframe, dtype = torch.int64)
'''

categorical_data = torch.tensor(categorical_data, dtype=torch.int64)
categorical_data[:10]

tensor([[3, 3, 0, 0, 2, 1],
        [3, 3, 0, 0, 2, 2],
        [3, 3, 0, 0, 2, 0],
        [3, 3, 0, 0, 1, 1],
        [3, 3, 0, 0, 1, 2],
        [3, 3, 0, 0, 1, 0],
        [3, 3, 0, 0, 0, 1],
        [3, 3, 0, 0, 0, 2],
        [3, 3, 0, 0, 0, 0],
        [3, 3, 0, 1, 2, 1]])

In [14]:
'''
pd.get_dummies : object형의 데이터들 기계학습을 위해서 데이터를 수치화 시킨다.

'''

outputs = pd.get_dummies(dataset.output)
outputs = outputs.values
outputs = torch.tensor(outputs).flatten()

print(categorical_data.shape)
print(outputs.shape)

torch.Size([1728, 6])
torch.Size([6912])


In [15]:
print("column 의 종류")
print(categorical_columns)

print("각 컬럼안에는 있는 데이터")
print([dataset[column].cat.categories for column in categorical_columns])

print("각 칼럼 안에 있는 데이터의 종류 갯수")
categorical_column_sizes = [len(dataset[column].cat.categories) for column in categorical_columns]
print(categorical_column_sizes)

'''
a//b 연산자 : a를 b로 나누었을 때 몫
'''
print("각 컬럼 안의 데이터 종류의 수, 그리고 그것을 2로 나눈 몫")
categorical_embedding_sizes = [(col_size, min(50, (col_size + 1) // 2)) for col_size in categorical_column_sizes]
print(categorical_embedding_sizes)

column 의 종류
['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']
각 컬럼안에는 있는 데이터
[Index(['high', 'low', 'med', 'vhigh'], dtype='object'), Index(['high', 'low', 'med', 'vhigh'], dtype='object'), Index(['2', '3', '4', '5more'], dtype='object'), Index(['2', '4', 'more'], dtype='object'), Index(['big', 'med', 'small'], dtype='object'), Index(['high', 'low', 'med'], dtype='object')]
각 칼럼 안에 있는 데이터의 종류 갯수
[4, 4, 4, 3, 3, 3]
각 컬럼 안의 데이터 종류의 수, 그리고 그것을 2로 나눈 몫
[(4, 2), (4, 2), (4, 2), (3, 2), (3, 2), (3, 2)]


In [16]:
# 총 데이터 수
total_records = 1728

# 테스트 데이터 수
test_records = int(total_records * .2)

# train data
categorical_train_data = categorical_data[:total_records - test_records]

# test data
categorical_test_data = categorical_data[total_records - test_records:total_records]

# output 혹은 타깃
train_outputs = outputs[:total_records - test_records]
test_outputs = outputs[total_records - test_records:total_records]

In [17]:
print(len(categorical_train_data))
print(len(train_outputs))
print(len(categorical_test_data))
print(len(test_outputs))

1383
1383
345
345


In [18]:
class Model(nn.Module):
    def __init__(self, embedding_size, output_size, layers, p=0.4):

        '''

        :param embedding_size: 임베딩 사이즈
        :param output_size:
        :param layers:
        :param p: 드롭 아웃 비율
        '''

        super().__init__()
        '''
        nn.ModuleList: 신경망의 모듈들의 리스트 index접근 가능
        nn.Embedding: embedding 층
        Embedding이란 말은 자연어 처리분야에서 매우 많이 등장하는 단어로 이산적, 범주형인 변수를 sparse한 one hot 인코딩 대신 연속적인 값을 가지는 벡터로 표현한는 방법
        '''
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(p)

        all_layers = []
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols

        # 신경망 생성
        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
            all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            # 각 층의 아웃풋의 크기는 다음 층의 인풋 사이즈
            input_size = i

        # 마지막 층은 regression 모델에서니까 linear 층
        all_layers.append(nn.Linear(layers[-1], output_size))
        self.layers = nn.Sequential(*all_layers)

    def forward(self, x_categorical):
        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        x = torch.cat(embeddings, 1)
        x = self.embedding_dropout(x)
        x = self.layers(x)
        return x

In [19]:
model = Model(categorical_embedding_sizes, 4, [200, 100, 50], p=0.4)
print(model)

Model(
  (all_embeddings): ModuleList(
    (0-2): 3 x Embedding(4, 2)
    (3-5): 3 x Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=4, bias=True)
  )
)


In [20]:
'''
nn.CrossEntropyLoss() : classification에서 주로쓰이는 softmax함수를 이용할 때 쓰는 loss function
'''
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [24]:
epochs = 500
aggregated_losses = []
train_outputs = train_outputs.to(device=device, dtype=torch.int64)
for i in range(epochs):
    i += 1
    '''
    RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument target in method wrapper_CUDA_nll_loss_forward)
    머신러닝이 같은 디바이스에 있어야한다.

    .cuda()로 gpu에 실행 가능하도록 하자
    '''
    y_pred = model(categorical_train_data).cuda()
    single_loss = loss_function(y_pred, train_outputs)
    aggregated_losses.append(single_loss)

    if i % 25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

    optimizer.zero_grad()
    single_loss.backward()
    optimizer.step()

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 1.60023487
epoch:  26 loss: 1.38107908
epoch:  51 loss: 1.30147707
epoch:  76 loss: 1.19144297
epoch: 101 loss: 1.08119500
epoch: 126 loss: 0.93182111
epoch: 151 loss: 0.81806785
epoch: 176 loss: 0.75787783
epoch: 201 loss: 0.69142503
epoch: 226 loss: 0.65746498
epoch: 251 loss: 0.63912427
epoch: 276 loss: 0.62003350
epoch: 301 loss: 0.60014272
epoch: 326 loss: 0.58700377
epoch: 351 loss: 0.59449834
epoch: 376 loss: 0.58636332
epoch: 401 loss: 0.57903302
epoch: 426 loss: 0.56916434
epoch: 451 loss: 0.56781286
epoch: 476 loss: 0.55858994
epoch: 500 loss: 0.5627627373


In [26]:
test_outputs = test_outputs.to(device=device, dtype=torch.int64)
'''
이와 같이 no_grad() with statement에 포함시키게 되면 Pytorch는 autograd engine을 꺼버린다. 이 말은 더 이상 자동으로 gradient를 트래킹하지 않는다는 말이 된다.
그러면 이런 의문이 들 수 있다. loss.backward()를 통해 backpropagation을 진행하지 않는다면 뭐 gradient를 게산하든지 말든지 큰 상관이 없는 것이 아닌가?

맞는 말이다. torch.no_grad()의 주된 목적은 autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함이다. 사실상 어짜피 안쓸 gradient인데 inference시에 굳이 계산할 필요가 없지 않은가?

그래서 일반적으로 inference를 진행할 때는 torch.no_grad() with statement로 감싼다는 사실을 알면 된다.


'''
with torch.no_grad():
    y_val = model(categorical_test_data).cuda()
    loss = loss_function(y_val, test_outputs)
print(f'Loss: {loss:.8f}')

Loss: 0.55101478


In [27]:
print(y_val[:5])

tensor([[ 2.4394,  1.3458, -2.8184, -2.7824],
        [ 2.1844,  1.2613, -2.6424, -2.5761],
        [ 2.6043,  1.6695, -3.1607, -3.0733],
        [ 3.2251,  1.7176, -4.0744, -4.0000],
        [ 2.4732,  1.2452, -3.0336, -3.1327]], device='cuda:0')


In [31]:
y_val = np.argmax(y_val.cpu(), axis=1)
print(y_val[:5])

tensor([0, 0, 0, 0, 0])


In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(test_outputs.cpu(), y_val))
print(classification_report(test_outputs.cpu(), y_val))
print(accuracy_score(test_outputs.cpu(), y_val))

[[258   1]
 [ 80   6]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86       259
           1       0.86      0.07      0.13        86

    accuracy                           0.77       345
   macro avg       0.81      0.53      0.50       345
weighted avg       0.79      0.77      0.68       345

0.7652173913043478
